In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("src/archive/user_reviews.csv")

df = df.drop(["isVerified", "isSuperReviewer", "hasSpoilers", "hasProfanity", "userRealm"], axis=1)
df = df.groupby("userId").head(5)

df.to_csv("src/processed/user_reviews.csv", index=False)